## 環境建立

In [ ]:
## requirements.txt
requests
pandas
jupyter
ipykernel


In [ ]:
# 把venv 命名為2025_kuangfu並註冊為jupyter notebook kernel

python -m ipykernel install --user --name=2025_kuangfu --display-name "Python (.venv 2025_kuangfu)"

# select jupyter notebook kernel


In [1]:
!which python
!python --version

/home/de-jung/projects/2025_kuangfu/.venv/bin/python
Python 3.12.3


## 以requests 把8張地圖載成kml，並下載政府救災資料為csv

In [9]:
# test_fetch_maps.ipynb 內容
import requests, os

os.makedirs("outputs/kml", exist_ok=True)

sources = {
    "1_zenuie": "https://zenuie.github.io/help_hub/",
    "2_support_hualien": "https://jstmonika.github.io/SupportHualien/",
    "3_rescue_map": "https://zkt.lax.mybluehost.me/rescue/rescuemap.html",
    "4_kiang": "https://tainan.olc.tw/p/guangfu250923/",
    "5_requirement_map": "https://www.google.com/maps/d/u/0/viewer?mid=1z3Lh3pKucPyjFiq-OVOQeAAJ_Y3viQk&ll=25.04114306979395%2C121.50900250000002&z=13",
    "6_yu_map": "https://www.google.com/maps/d/u/0/embed?mid=1qOHK91tv68NacIN1GVTDYKn10ojb-t8&ll=23.6409918967309%2C121.43978255&z=11",
    "7_sam_map": "https://www.google.com/maps/d/u/1/viewer?hl=zh-TW&mid=1rH9sGuSh-Oqzcrtj7wgaenDWDOfHH-c&ll=23.79589192204332%2C121.4612344595703&z=11",
    "8_lin_map": "https://www.google.com/maps/d/u/0/viewer?hl=zh-TW&ll=23.669393075256767%2C121.4317209625592&z=16&mid=1l-TUsEbH5ahJBbXxCnKL2oCFB39hYw8",
    "gov_sheet": "https://docs.google.com/spreadsheets/d/1s-6z7kqsRLUEsmetdquoBDbZugwYmHiRKYeWCdc-gP4/export?format=csv",
}

def fetch_file(name, url):
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        ext = ".csv" if "format=csv" in url else ".kml"
        path = f"outputs/kml/{name}{ext}"
        with open(path, "wb") as f:
            f.write(r.content)
        print(f"✅ {name} saved → {path} ({len(r.content)} bytes)")
    except Exception as e:
        print(f"❌ {name} failed: {e}")

for n, u in sources.items():
    fetch_file(n, u)


✅ 1_zenuie saved → outputs/kml/1_zenuie.kml (709 bytes)
✅ 2_support_hualien saved → outputs/kml/2_support_hualien.kml (823 bytes)
❌ 3_rescue_map failed: 406 Client Error: Not Acceptable for url: https://zkt.lax.mybluehost.me/rescue/rescuemap.html
✅ 4_kiang saved → outputs/kml/4_kiang.kml (31275 bytes)
✅ 5_requirement_map saved → outputs/kml/5_requirement_map.kml (127877 bytes)
✅ 6_yu_map saved → outputs/kml/6_yu_map.kml (534557 bytes)
✅ 7_sam_map saved → outputs/kml/7_sam_map.kml (177384 bytes)
✅ 8_lin_map saved → outputs/kml/8_lin_map.kml (226028 bytes)
✅ gov_sheet saved → outputs/kml/gov_sheet.csv (22376 bytes)


In [11]:
## FOR 3_rescue_map
import requests, re, os

url = "https://zkt.lax.mybluehost.me/rescue/rescuemap.html"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0 Safari/537.36"
}

# 下載 HTML
resp = requests.get(url, headers=headers, timeout=30)
resp.raise_for_status()
html = resp.text

# 建立資料夾
os.makedirs("outputs/kml", exist_ok=True)

# 嘗試找是否有 .kml / .geojson 的 URL
matches = re.findall(r'https?://[^\s"\']+\.(?:kml|geojson)', html)

if matches:
    for i, link in enumerate(matches):
        print(f"找到地圖資料來源 {i+1}: {link}")
        r2 = requests.get(link, headers=headers, timeout=30)
        r2.raise_for_status()
        fname = f"outputs/kml/3_rescue_map_{i+1}.kml" if link.endswith(".kml") else f"outputs/kml/3_rescue_map_{i+1}.geojson"
        with open(fname, "wb") as f:
            f.write(r2.content)
        print(f"✅ 已儲存: {fname}")
else:
    # 如果沒有找到 .kml / .geojson，就先把 HTML 存起來手動檢查
    with open("outputs/kml/3_rescue_map_page.html", "w", encoding="utf-8") as f:
        f.write(html)
    print("⚠️ 沒找到 KML/GeoJSON 連結，已保存 HTML 供檢查: outputs/kml/3_rescue_map_page.html")


⚠️ 沒找到 KML/GeoJSON 連結，已保存 HTML 供檢查: outputs/kml/3_rescue_map_page.html


In [12]:
# 3_rescue_map_page.html 內容
import requests, json, os

API_URL = "https://script.google.com/macros/s/AKfycbxjjQOQXrMR64rQ4YNMf0NLLEC6p8H21XiRjlYMijvfR9SPvT1x4tlCe2S4cjfX9m3xgg/exec"

resp = requests.get(API_URL, timeout=30)
resp.raise_for_status()
data = resp.json()

os.makedirs("outputs/kml", exist_ok=True)

# 存成 JSON 以便檢查
with open("outputs/kml/3_rescue_map.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"✅ 已下載 JSON, 共 {len(data)} 筆")


✅ 已下載 JSON, 共 10 筆


## kml to json

In [13]:
import xml.etree.ElementTree as ET
import json
import unicodedata
import re
import os

def kml_color_to_hex(kml_color):
    """Converts a KML color string (aabbggrr) to a standard web hex color (#RRGGBB)."""
    if not kml_color or len(kml_color) != 8:
        return None
    red = kml_color[6:8]
    green = kml_color[4:6]
    blue = kml_color[2:4]
    return f'#{red}{green}{blue}'.upper()

def parse_kml_styles(root, ns):
    """Parses <Style> and <StyleMap> elements and returns a lookup dictionary mapping style URLs to icon and color properties."""
    style_map = {}

    # Parse <Style>
    for style in root.findall(f'.//{ns}Style'):
        style_id = style.get('id')
        if not style_id:
            continue
        details = {'icon': None, 'color': None}

        icon_href = style.find(f'{ns}IconStyle/{ns}Icon/{ns}href')
        if icon_href is not None and icon_href.text:
            details['icon'] = icon_href.text.strip()

        color_element = style.find(f'{ns}IconStyle/{ns}color')
        if color_element is None:
            color_element = style.find(f'{ns}LineStyle/{ns}color') or style.find(f'{ns}PolyStyle/{ns}color')

        if color_element is not None and color_element.text:
            details['color'] = kml_color_to_hex(color_element.text.strip())

        style_map[f'#{style_id}'] = details

    # Parse <StyleMap>
    for style_map_element in root.findall(f'.//{ns}StyleMap'):
        map_id = style_map_element.get('id')
        if not map_id:
            continue
        normal_pair = style_map_element.find(f'{ns}Pair[{ns}key="normal"]/{ns}styleUrl')
        if normal_pair is not None and normal_pair.text:
            style_map[f'#{map_id}'] = style_map.get(normal_pair.text.strip(), {'icon': None, 'color': None})

    return style_map

def extract_placemark_data(file_path):
    """Parses a KML file and extracts placemark info into JSON-like structure."""
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
    except ET.ParseError:
        with open(file_path, 'r', encoding='utf-8') as f:
            xml_content = f.read()
        xml_content = "".join(ch for ch in xml_content if unicodedata.category(ch)[0] != 'C' or ch in ('\t', '\n', '\r'))
        try:
            root = ET.fromstring(xml_content)
        except ET.ParseError as e:
            print(f"❌ Failed to parse {file_path}: {e}")
            return []
    except FileNotFoundError as e:
        print(f"❌ File not found: {file_path}")
        return []

    placemarks_data = []
    ns = '{http://www.opengis.net/kml/2.2}'
    style_lookup = parse_kml_styles(root, ns)

    for placemark in root.findall(f'.//{ns}Placemark'):
        name_element = placemark.find(f'{ns}name')
        name = name_element.text.strip() if name_element is not None and name_element.text else 'No Name'

        description_element = placemark.find(f'{ns}description')
        description = description_element.text.strip() if description_element is not None and description_element.text else 'No Description'

        style_url_element = placemark.find(f'{ns}styleUrl')
        style_url = style_url_element.text.strip() if style_url_element is not None and style_url_element.text else None
        style_details = style_lookup.get(style_url, {'icon': None, 'color': None})

        # Geometry
        point = placemark.find(f'.//{ns}Point')
        linestring = placemark.find(f'.//{ns}LineString')
        polygon = placemark.find(f'.//{ns}Polygon')

        geom_type = 'Unknown'
        coordinates_text = None
        if point is not None:
            geom_type = 'Point'
            coordinates_element = point.find(f'.//{ns}coordinates')
            if coordinates_element is not None:
                coordinates_text = coordinates_element.text
        elif linestring is not None:
            geom_type = 'LineString'
            coordinates_element = linestring.find(f'.//{ns}coordinates')
            if coordinates_element is not None:
                coordinates_text = coordinates_element.text
        elif polygon is not None:
            geom_type = 'Polygon'
            coordinates_element = polygon.find(f'.//{ns}LinearRing/{ns}coordinates')
            if coordinates_element is not None:
                coordinates_text = coordinates_element.text

        coords = []
        if coordinates_text:
            coord_tuples = re.findall(r'(-?\d+\.\d+),(-?\d+\.\d+)(?:,-?\d+\.?\d*)?', coordinates_text.strip())
            for lon_str, lat_str in coord_tuples:
                try:
                    lon = float(lon_str)
                    lat = float(lat_str)
                    coords.append([lon, lat])
                except ValueError:
                    continue

        if coords:  # ✅ Only keep placemarks with coordinates
            placemarks_data.append({
                'name': name,
                'description': description,
                'type': geom_type,
                'coordinates': coords,
                'color': style_details['color'],
                'icon': style_details['icon']
            })

    return placemarks_data

# === 批次處理 outputs/kml/ 資料夾中的 KML 檔案 ===
input_folder = "outputs/kml"
output_folder = "outputs/kml"

kml_files = [f for f in os.listdir(input_folder) if f.endswith(".kml")]

for kml_file in kml_files:
    input_path = os.path.join(input_folder, kml_file)
    output_path = os.path.join(output_folder, kml_file.replace(".kml", ".json"))

    data = extract_placemark_data(input_path)

    if data:
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        print(f"✅ Converted {kml_file} → {output_path} ({len(data)} placemarks)")
    else:
        print(f"⚠️ No placemarks found in {kml_file}")


❌ Failed to parse outputs/kml/7_sam_map.kml: not well-formed (invalid token): line 1, column 31
⚠️ No placemarks found in 7_sam_map.kml
❌ Failed to parse outputs/kml/8_lin_map.kml: not well-formed (invalid token): line 1, column 31
⚠️ No placemarks found in 8_lin_map.kml
❌ Failed to parse outputs/kml/5_requirement_map.kml: not well-formed (invalid token): line 1, column 31
⚠️ No placemarks found in 5_requirement_map.kml
❌ Failed to parse outputs/kml/6_yu_map.kml: not well-formed (invalid token): line 1, column 31
⚠️ No placemarks found in 6_yu_map.kml
❌ Failed to parse outputs/kml/2_support_hualien.kml: mismatched tag: line 11, column 2
⚠️ No placemarks found in 2_support_hualien.kml
❌ Failed to parse outputs/kml/4_kiang.kml: not well-formed (invalid token): line 567, column 18
⚠️ No placemarks found in 4_kiang.kml
❌ Failed to parse outputs/kml/1_zenuie.kml: syntax error: line 1, column 0
⚠️ No placemarks found in 1_zenuie.kml
